In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

import numpy as np
from log import log

In [5]:
import sys
import json

In [ ]:
from pprint import pprint

In [24]:
sql = """
with temp as (
select 	a.id emr_id,
		a.doctor_id doctor_id,
		a.patient_id patient_id,
		p_patient.sex,
		date(a.finished at time zone 'UTC') survey_date,
		p_patient.birthday,
		(select trim(s.name)
		    from specialty s
		    inner join doctor_specialty ds
		       on ds.specialty_id = s.id
		    inner join doctor_specialty_treatment dst
		       on dst.doctor_specialty_id = ds.id
		    where dst.treatment_id = a.treatment_id
		    limit 1 -- из-за одной криво заведенной записи.
		) specialization_code,
		--
		a.report_comment,
		review.rating review_rating,
		review."comment" review_comment, 
		a."source" consultation_source,
		partner.name
--
from appointment a 
--
left join patient on a.patient_id = patient.id 
left join person p_patient on patient.person_id = p_patient.id
--
left join promotion promo on a.promotion_id = promo.id 
left join partner on promo.partner_id = partner.id 
--
left join review on a.id = review.appointment_id 
--
where a.finished at time zone 'UTC' > '2020-05-01'
and a.finished at time zone 'UTC' < '2020-06-01'
and a.good
  and not (
    a.patient_id in (
        select patient_id
        from patient_categories pcat
        inner join reference rf
           on rf.id=pcat.reference_id
        where rf.code='TEST'
        )
    or lower(a.report_comment)='тест'
    )
--
and source='QUEUED'
--
)
select 	emr_id,
		doctor_id,
		patient_id,
		case when sex is null
			then 'Unknown'
			else sex
		end sex,
		survey_date,
		birthday,
		consultation_source,
		case when specialization_code = 'Терапевт'
					then 'therapist'
			 when specialization_code = 'Педиатр'
			 		then 'pediatrist'
			 else 'Other'
		end specialization_code,
		--
		review_comment,
		review_rating,
        report_comment
from temp 
where name = 'DocDoc'
order by survey_date
    
    
"""

In [25]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-06-15 13:10:23 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-06-15 13:10:23 creating ssh tunnel to 172.16.100.19 as root...
2020-06-15 13:10:23 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 42069
2020-06-15 13:10:23 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-06-15 13:10:23 @telemed query:
                    with temp as (
				    select 	a.id emr_id,
				    		a.doctor_id doctor_id,
				    		a.patient_id patient_id,
				    		p_patient.sex,
				    		date(a.finished at time zone 'UTC') survey_date,
				    		p_patient.birthday,
				    		(select trim(s.name)
				    		    from specialty s
				    		    inner join doctor_specialty ds
				    		       on ds.specialty_id = s.id
				    		    inner join doctor_specialty_treatment dst
				    		       on dst.doctor_specialty_id = ds.id

In [26]:
data

,emr_id,doctor_id,patient_id,sex,survey_date,birthday,consultation_source,specialization_code,review_comment,review_rating,report_comment
0,176318,8313,3542498,Unknown,2020-05-01,1984-10-23,QUEUED,therapist,None,NaN,"Жалобы: боль в спине , отдаёт в бок и живот. А..."
1,176102,7593,1631838,Unknown,2020-05-01,1992-08-14,QUEUED,therapist,None,NaN,1. Анамнез: пациентка болеет 1 неделю. Вызывал...
2,176316,7267,2961564,Unknown,2020-05-01,1993-09-29,QUEUED,therapist,None,NaN,Солдатова Вера Александровна\n26 лет\nЖалобы: ...
3,176262,8279,3535284,FEMALE,2020-05-01,1987-12-04,QUEUED,therapist,None,NaN,"Жалобы на дискомфорт, заложенность в горле, те..."
4,175096,6377,1020683,FEMALE,2020-05-01,1990-01-14,QUEUED,therapist,Спасибо,5.0,"Жалобы: на сыпь на теле, конечностях, полости ..."
...,...,...,...,...,...,...,...,...,...,...,...
2341,240672,7687,3561741,Unknown,2020-05-31,1955-01-18,QUEUED,therapist,None,NaN,Рекомендовано : \n-очная консультация гастроэн...
2342,240831,505,38488,FEMALE,2020-05-31,1979-11-23,QUEUED,pediatrist,None,NaN,"На основании анамнеза, жалоб, учитывая возраст..."
2343,240637,4010,2579011,FEMALE,2020-05-31,1993-09-28,QUEUED,therapist,None,NaN,с целью увеличения лактации возможен прием фит...
2344,241825,606,3765584,Unknown,2020-05-31,1992-09-17,QUEUED,therapist,None,NaN,"Исходя из предоставленной информации, можно пр..."


In [27]:
type(data['survey_date'][0])

datetime.date

In [50]:
def dataconverter(o):
    if isinstance(o, datetime.date):
        return o.__str__()
        #return "{}-{}-{}".format(o.year, o.month, o.day)

In [51]:
dataconverter(data['survey_date'][0])

'2020-05-01'

In [52]:
result_file: str = sys.path[0] + '/consultations.json'
with open(result_file, 'w', encoding='utf-8') as f:
    json.dump(data.to_dict('records'), f, ensure_ascii=False, default=dataconverter)

In [53]:
with open(result_file, encoding='utf-8') as f:
    s = json.load(f)
pprint(s[0:2])

[{'birthday': '1984-10-23',
  'consultation_source': 'QUEUED',
  'doctor_id': 8313,
  'emr_id': 176318,
  'patient_id': 3542498,
  'report_comment': 'Жалобы: боль в спине , отдаёт в бок и живот. Анамнез : '
                    'боль появилась после физической нагрузки ( тренировка с '
                    'гантелями). Диагностическая гипотеза: учитывая жалобы и '
                    'анамнез, у пациента растяжение мышцы спины.     '
                    'Рекомендации: диклофенак по 1 таб 2 раза в день.\r\n'
                    '3. мазь диклофенак на область спины 2-3 раза в день. '
                    'Массаж местно, тепло местно. Исключить физическую '
                    'нагрузку . . В случае ухудшения состояния , консультация '
                    'невролога.',
  'review_comment': None,
  'review_rating': nan,
  'sex': 'Unknown',
  'specialization_code': 'therapist',
  'survey_date': '2020-05-01'},
 {'birthday': '1992-08-14',
  'consultation_source': 'QUEUED',
  'doctor_id': 7593,
  